In [2]:
#dependencies
import requests
import time
import datetime as dt
import csv
import re
import pandas as pd
from bs4 import BeautifulSoup as BS

In [ ]:
#Data collection for a single product from a URL
#Part 1 of the string

#Preparing CSV that information will be added to
productHeader = ['title', 'genre', 'price', 'quantity', 'in_stock', 'UPC', 'product_type', 'date_accessed']
with open('ProductInformation.csv', 'w', newline = '', encoding = 'UTF8') as a:
    writer = csv.writer(a)
    writer.writerow(productHeader)


def productExtract(URL):
    #Preparing URL for parsing
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get (URL, headers = headers)
    soup = BS(page.content, 'html.parser')


    #Collecting title and genre with bs4
    g = soup.find(class_ = 'breadcrumb').get_text()
    g_list = g.split("\n")

    #Selecting title and genre from breadcrump list
    title = g_list[-2]
    genre = g_list[-4]


    #Collecting price
    price = soup.find(class_ = 'price_color').get_text()


    #Collecting quantity and availability (in_stock)
    availability = soup.find(class_ = 'instock availability').get_text()

    #Selecting quantity by selecting only numeric values
    quantity = re.findall(r'\d+', availability)
    quantity = quantity.pop()

    #Using if/else statement to determine availability
    quantity = int(quantity)
    if quantity <= 0:
        in_stock = 'no'
    else:
        in_stock = 'yes'


    #Collecting UPC and product type from informational table    
    t_info = soup.find(class_ = 'table table-striped').get_text()
    t_info = t_info.split()

    #Selecting UPC and product type
    UPC = t_info[0]
    UPC = UPC[2:]
    product_type = t_info[2]
    product_type = product_type[4:]


    #Determining when the data was accessed
    date_accessed = dt.datetime.now()
    date_accessed = date_accessed.strftime("%Y-%m-%d %H:%M")

    #Display results
    results = [title, genre, price, quantity, in_stock, UPC, product_type, date_accessed]
    print(results)
    with open('ProductInformation.csv', 'a+', newline = '', encoding = 'UTF8') as a:
        writer = csv.writer(a)
        writer.writerow(results)
        
productExtract('https://books.toscrape.com/catalogue/in-her-wake_980/index.html')

In [ ]:
#Function for collecting the URL of every product in a page
#Part 2 of the string

#Preparing CSV that links will be added to
header = ['title','link', 'date_accessed']
with open('ProductURL.csv', 'w', newline = '', encoding = 'UTF8') as b:
    writer = csv.writer(b)
    writer.writerow(header)

def extract(URL):
    
    #Preparing url for parsing   
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get (URL, headers = headers)
    soup = BS(page.content, 'html.parser')
       
    #Collecting all links on URL on a list
    links = soup.find_all('a')
    links = list(links)

    #Categories of urls collected
    book_url = []
    book_url_2 = []
    category_url = []
    page_url = []
    index_url = []

    #Categorizing links and sorting them into lists
    for x in links:
        #Extract links from list and turning them into strings
        href = x["href"]
        href = str(href)
        
        #Filter out non-book links
        if 'category/books' in href: 
            category_url.append(href)
        
        elif '../index.html' in href:
            index_url.append(href)
            
        #Filter out non-book links
        elif 'page-' in href:
            nextPage = href

        #Adding book links to a list
        elif 'index.html' not in href[0:10]:
            book_url_2.append(href)

    #Collecting unique links for books, each book has two links on each page    
    for hrefLink in book_url_2:
        if hrefLink not in book_url:
            book_url.append(hrefLink)
            
            end = hrefLink.index('_')
            title = hrefLink[:end]
            title = title.replace('-', ' ')
            title = title.title()

            date_accessed = dt.datetime.now()
            date_accessed = date_accessed.strftime("%Y-%m-%d %H:%M")
            
            row = (title, hrefLink, date_accessed)
            with open('ProductURL.csv', 'a+', newline = '\n', encoding = 'UTF8') as b:
                writer = csv.writer(b)
                writer.writerow(row)              
    
extract('https://books.toscrape.com/catalogue/page-1.html')


print('your file is ready mr. tanabe ❤️ ')

In [ ]:
#Function for moving pages
#Part 3 of the string


def extract(URL):
    while True:
        currentPage = URL[42:-5]
        currentPage = int(currentPage)
        print('Currently parsing Page ', currentPage)

        #Preparing url for parsing   
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
        page = requests.get (URL, headers = headers)
        soup = BS(page.content, 'html.parser')

        #Collecting all links on URL on a list
        links = soup.find_all('a')
        links = list(links)

        #Categories of urls collected
        page_url = []

        #Categorizing links and sorting them into lists
        for x in links:
        #Extract links from list and turning them into strings
            href = x["href"]
            href = str(href)

        #Categorizing links and sorting them into lists
        for x in links:
            #Find the link to the next page
            if 'page-' in href:
                nextPage = href #NOTE There are four links that direct to a different page on the site. The last one always directs to the next page. It will get rewritten'
            else:
                pass

        nextPage = nextPage[5:-5]
        URL = 'https://books.toscrape.com/catalogue/page-' + nextPage + '.html'
        print('the next page is ' + nextPage)
        if currentPage == 10:
            break
    return currentPage

extract('https://books.toscrape.com/catalogue/page-1.html')


print('your file is ready mr. tanabe ❤️ ')

In [ ]:
#First version of the file put together with comments from Mr. Alialy

#Preparing CSV that links will be added to
header = ['title','link', 'date_accessed']
with open('ProductURL.csv', 'w', newline = '', encoding = 'UTF8') as b:
    writer = csv.writer(b)
    writer.writerow(header)

#Preparing CSV that information will be added to
productHeader = ['title', 'genre', 'price', 'quantity', 'in_stock', 'UPC', 'product_type', 'date_accessed']
with open('ProductInformation.csv', 'w', newline = '', encoding = 'UTF8') as a:
    writer = csv.writer(a)
    writer.writerow(productHeader)
    
def productExtract(URL):
    #Preparing URL for parsing
    URL = 'https://books.toscrape.com/catalogue/' + URL
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get (URL, headers = headers)
    soup = BS(page.content, 'html.parser')


    #Collecting title and genre with bs4
    g = soup.find(class_ = 'breadcrumb').get_text()
    g_list = g.split("\n")

    #Selecting title and genre from breadcrump list
    title = g_list[-2]
    genre = g_list[-4]
#kyle - remove whitespace from g list. ALSO pull when to stop page from the actual page itself


    #Collecting price
    price = soup.find(class_ = 'price_color').get_text()


    #Collecting quantity and availability (in_stock)
    availability = soup.find(class_ = 'instock availability').get_text()

    #Selecting quantity by selecting only numeric values
    quantity = re.findall(r'\d+', availability)
    quantity = quantity.pop()

    #Using if/else statement to determine availability
    quantity = int(quantity)
    if quantity <= 0:
        in_stock = 'no'
    else:
        in_stock = 'yes'


    #Collecting UPC and product type from informational table    
    t_info = soup.find(class_ = 'table table-striped').get_text()
    t_info = t_info.split()

    #Selecting UPC and product type
    UPC = t_info[0]
    UPC = UPC[2:]
    product_type = t_info[2]
    product_type = product_type[4:]
    #MAKE THIS MORE EXPLICIT, Pull table then pull based on tags


    #Determining when the data was accessed
    date_accessed = dt.datetime.now()
    date_accessed = date_accessed.strftime("%Y-%m-%d %H:%M")

    #Display results
    results = [title, genre, price, quantity, in_stock, UPC, product_type, date_accessed]
    with open('ProductInformation.csv', 'a+', newline = '', encoding = 'UTF8') as a:
        writer = csv.writer(a)
        writer.writerow(results)
    
def extract(URL):
#add a max index through a for loop
#NEVER USE THE SAME THING
    while True:
        currentPage = URL[42:-5]
        currentPage = int(currentPage)
        print(currentPage)
        #currentPage = int(currentPage)
        print('Currently parsing Page ', currentPage)

        #Preparing url for parsing   
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
        page = requests.get (URL, headers = headers)
        soup = BS(page.content, 'html.parser')

        #Collecting all links on URL on a list
        links = soup.find_all('a')
        links = list(links)

        #Categories of urls collected
        book_url = []
        book_url_2 = []
        category_url = []
        page_url = []
        index_url = []

        #Categorizing links and sorting them into lists
        #REWRITE ALL DIS SHIT, USE CLASS TO ONLY SELECT ITEMS
        for x in links:
        #Extract links from list and turning them into strings
            href = x["href"]
            href = str(href)

            #Find the link to the next page
            if 'page-' in href:
                nextPage = href #NOTE There are four links that direct to a different page on the site. The last one always directs to the next page. It will get rewritten'

            #Create list of categories
            elif 'category/books' in href: 
                category_url.append(href)

            #Put indices in one list
            elif '../index.html' in href:
                index_url.append(href)

            #Adding book links to a list
            elif 'index.html' not in href[0:10]:
                book_url_2.append(href)

        #Collecting unique links for books, each book has two links on each page    
        for hrefLink in book_url_2:
            if hrefLink not in book_url:
                productExtract(hrefLink)
                book_url.append(hrefLink)

                end = hrefLink.index('_')
                title = hrefLink[:end]
                title = title.replace('-', ' ')
                title = title.title()

                date_accessed = dt.datetime.now()
                date_accessed = date_accessed.strftime("%Y-%m-%d %H:%M")

                row = (title, hrefLink, date_accessed)
                with open('ProductURL.csv', 'a+', newline = '\n', encoding = 'UTF8') as b:
                    writer = csv.writer(b)
                    writer.writerow(row)
                    

        nextPage = currentPage + 1
        nextPage2 = str(nextPage)
        URL = 'https://books.toscrape.com/catalogue/page-' + nextPage2 + '.html'
        
        #do if theres no next, then stop
        if nextPage == 51:
            print('Parsing successful! Please check your new CSV files.')
            break
        else: 
            print('The next page is: ')
            continue       
    
page2 = 'https://books.toscrape.com/catalogue/page-1.html'
extract(page2)


In [31]:
#Version two of part 1
URL = 'https://books.toscrape.com/catalogue/frankenstein_20/index.html'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

def productExtract(url):
    page = requests.get(URL, headers = headers)
    soup = BS(page.content, 'html.parser')

    #Collect the title
    title = soup.html.title.text
    title = title[:title.index('|')].strip()

    #Collect the genre, which is located in a breadcrumb list
    genre = soup.find('ul').text
    #Create list of breadcrumb items
    genre = genre.split('\n')
    genre = list(filter(None, genre))[-2]

    #Collecting price
    price = soup.find(class_ = 'price_color').get_text()


    #Collecting quantity and availability (in_stock)
    #Returns a statement ie: 'In stock (21) available'
    availability = soup.find(class_ = 'instock availability').get_text()

    #Selecting quantity by filtering numeric values into a list
    #This puts numeric values into a list
    quantity = re.findall(r'\d+', availability)
    #Removes the quantity + changes to integer type
    quantity = int(quantity.pop())


    #Using if/else statement to determine availability status
    quantity = int(quantity)
    if quantity <= 0:
        in_stock = 'no'
    else:
        in_stock = 'yes'

    #Selecting UPC and product type
    UPC = soup.html.table.td.text
    product_type = list(soup.find_all('td'))[1].text

    #Determining when the data was accessed
    date_accessed = dt.datetime.now()
    date_accessed = date_accessed.strftime("%Y-%m-%d %H:%M")

    #Display results
    results = [title, genre, price, quantity, in_stock, UPC, product_type, date_accessed]

    with open('ProductInformation.csv', 'a+', newline = '', encoding = 'UTF8') as a:
        writer = csv.writer(a)
        writer.writerow(results)
        
productExtract(URL)


In [23]:
#Version 2 of parts 2 and 3 put together
URL = 'https://books.toscrape.com/catalogue/page-1.html'
page = requests.get (URL)
soup = BS(page.content, 'html.parser')

#Preparing CSV that links will be added to
header = ['title','link', 'dateAccessed']
with open('ProductURL.csv', 'w', newline = '', encoding = 'UTF8') as b:
    writer = csv.writer(b)
    writer.writerow(header) 
   
#Pass URL through this function to collect all links in the function
def pageCollect(url):
    page = requests.get (URL)
    soup = BS(page.content, 'html.parser')
    
    bookContainers = soup.find_all("li", {'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})

    for container in bookContainers:
        containerTitle = container.find('a').find('img').get('alt')
        
        href = container.find('a').get('href')
        bookLink = 'https://books.toscrape.com/catalogue/'

        dateAccessed = dt.datetime.now().strftime("%Y-%m-%d %H:%M")

        row = (containerTitle, bookLink, dateAccessed)
        with open('ProductURL.csv', 'a+', newline = '\n', encoding = 'UTF8') as b:
            writer = csv.writer(b)
            writer.writerow(row)
        
#For finding the total number of pages in the online store
pageNum = soup.find("ul", {"class": "pager"}).text.strip()
pageTotal = int(pageNum[pageNum.index('of') + 3 : pageNum.index('next')])
pageCurrent = 0

#For moving between pages
#GENERATES A NEW URL
# for i in range(1, pageTotal + 1):
for i in range(1, 5):
    URL = 'https://books.toscrape.com/catalogue/page-' + str(i) + '.html'
    pageCollect(URL)
    
    

In [44]:
#Final string: version 2 of the whole script

#Creating CSV that links collected from each store page will be added to
header = ['title','link', 'dateAccessed']
with open('ProductURL.csv', 'w', newline = '', encoding = 'UTF8') as b:
    writer = csv.writer(b)
    writer.writerow(header)
    
#Creating CSV that information from each product will be added to
productHeader = ['title', 'genre', 'price', 'quantity', 'inStock', 'UPC', 'productType', 'dateAccessed']
with open('ProductInformation.csv', 'w', newline = '', encoding = 'UTF8') as a:
    writer = csv.writer(a)
    writer.writerow(productHeader)

#Function for extracting product information
def productExtract(url):
    page = requests.get(url, headers = headers)
    soup = BS(page.content, 'html.parser')

    #Select the title
    title = soup.html.title.text
    title = title[:title.index('|')].strip()

    #Select the genre, which is only located in a breadcrumb list
    genre = soup.find('ul').text
    #Create list of breadcrumb items
    genre = genre.split('\n')
    genre = list(filter(None, genre))[-2]

    #Selecting price
    price = soup.find(class_ = 'price_color').get_text()

    #Selecting quantity and availability (inStock)
    #Returns a statement ie: 'In stock (21) available'
    availability = soup.find(class_ = 'instock availability').text

    #Selecting quantity by filtering numeric values into a list
    quantity = re.findall(r'\d+', availability)[0]

    #Using quantity to determine availability
    quantity = int(quantity)
    if quantity <= 0:
        inStock = 'no'
    else:
        inStock = 'yes'

    #Selecting UPC and product type
    UPC = soup.html.table.td.text
    productType = list(soup.find_all('td'))[1].text

    #Determine when data was accessed
    dateAccessed = dt.datetime.now().strftime("%Y-%m-%d %H:%M")

    #Write results onto CSV with product information
    results = [title, genre, price, quantity, inStock, UPC, productType, dateAccessed]
    with open('ProductInformation.csv', 'a+', newline = '', encoding = 'UTF8') as a:
        writer = csv.writer(a)
        writer.writerow(results)
        
#Function for collecting URL for every product in store page
def pageCollect(url2):
    page = requests.get(url2)
    soup = BS(page.content, 'html.parser')
    
    #Collect all containers containing product title and link
    bookContainers = soup.find_all("li", {'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})

    for container in bookContainers:
        #Select title
        containerTitle = container.find('a').find('img').get('alt')
        
        #Select link
        href = container.find('a').get('href')
        bookLink = 'https://books.toscrape.com/catalogue/' + href
        
        #Determine when data was accessed
        dateAccessed = dt.datetime.now().strftime("%Y-%m-%d %H:%M")
        
        #Write results onto CSV with product links
        row = (containerTitle, bookLink, dateAccessed)
        with open('ProductURL.csv', 'a+', newline = '\n', encoding = 'UTF8') as b:
            writer = csv.writer(b)
            writer.writerow(row)
        productExtract(bookLink)
        
#Finds total number of product store pages
#Returns string ie: 'Page 1 of x (x = number of pages)'
pageNum = soup.find("ul", {"class": "pager"}).text.strip()
pageTotal = int(pageNum[pageNum.index('of') + 3 : pageNum.index('next')])

#For moving between pages
#GENERATES A NEW URL
for i in range(1, pageTotal + 1):
    print('Currently parsing through page ' + str(i) + '.')
    URL = 'https://books.toscrape.com/catalogue/page-' + str(i) + '.html'
    pageCollect(URL)
    

Currently parsing through page 1.
Currently parsing through page 2.
Currently parsing through page 3.
Currently parsing through page 4.
Currently parsing through page 5.
Currently parsing through page 6.
Currently parsing through page 7.
Currently parsing through page 8.
Currently parsing through page 9.
Currently parsing through page 10.
Currently parsing through page 11.
Currently parsing through page 12.
Currently parsing through page 13.
Currently parsing through page 14.
Currently parsing through page 15.
Currently parsing through page 16.
Currently parsing through page 17.
Currently parsing through page 18.
Currently parsing through page 19.
Currently parsing through page 20.
Currently parsing through page 21.
Currently parsing through page 22.
Currently parsing through page 23.
Currently parsing through page 24.
Currently parsing through page 25.
Currently parsing through page 26.
Currently parsing through page 27.
Currently parsing through page 28.
Currently parsing through pag